<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
# Parameters
msgs = "Ran from Airflow at 2021-06-20T15:27:10.186128+00:00!"


In [2]:
import pandas as pd
import psycopg2
import os
import json
import datetime

In [3]:
with open("/home/anas/notebook/UseCase/config.json") as f:
    config = json.load(f)
print("Config:",config)

Config: {'database': 'datamart', 'user': 'postgres', 'password': 'intelrebel', 'host': 'localhost', 'port': '5432'}


In [4]:
#read config file in input
with open("/home/anas/notebook/UseCase/input_config.txt") as f:
    input_file=f.readline()
print("will process:",input_file)

will process: /home/anas/notebook/UseCase/input/meetingAttendanceReport(General)(2).csv


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
meta_data=[]
log_call=[]
lines=[]
with open(input_file,encoding="utf-16") as f:
    lines=f.readlines()
meta_data=lines[0:5]
log_call=lines[6:]

FileNotFoundError: [Errno 2] No such file or directory: '/home/anas/notebook/UseCase/input/meetingAttendanceReport(General)(2).csv'

In [ ]:
course = "DWH"
course

In [ ]:
with open(f"/home/anas/notebook/UseCase/output/log_{input_file.split('/')[-1]}","w") as f:
    for value in log_call:
        f.write(value)
    f.close()
df_log = pd.read_csv(f"/home/anas/notebook/UseCase/output/log_{input_file.split('/')[-1]}",sep="\t")
df_log

In [ ]:
start_str = meta_data[3].replace("Heure de début de la réunion\t","").replace("\n","").replace(" à","")
end_str   = meta_data[4].replace("Heure de fin\t","").replace("\n","").replace(" à","")
start_course = datetime.datetime.strptime(start_str, '%d/%m/%Y %H:%M:%S')
end_course = datetime.datetime.strptime(end_str, '%d/%m/%Y %H:%M:%S')

In [ ]:
mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
mycursor = mydb.cursor()
table="fact_table"
mycursor.execute(f"SELECT * FROM information_schema.columns WHERE table_schema = 'biabsence' AND table_name   = '{table}' ;")
myresult = mycursor.fetchall()
for value in myresult:
    print(value[3],":",value[7])

In [ ]:
def add_student(name,gender,email,course,major):
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"SELECT * FROM students where email ='{email}'")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO students (name,gender,email,course,major) VALUES (%s,%s,%s,%s,%s)"
        val = (name,gender,email,course,major)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"SELECT * FROM students where email ='{email}'")
        myresult = mycursor.fetchall()
        return myresult[0][0]
    
def add_professor(name,email):
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"SELECT * FROM professors where email ='{email}'")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO professors (name,email) VALUES (%s,%s)"
        val = (name,email)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"SELECT * FROM professors where email ='{email}'")
        myresult = mycursor.fetchall()
        return myresult[0][0]
    
def add_time(hour,day,month,year,date):
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"SELECT * FROM Time where year = {year} and month={month} and day={day} and hour={hour}")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO time (hour,day,month,year,date) VALUES (%s,%s,%s,%s,%s)"
        val = (hour,day,month,year,date)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"SELECT * FROM time where year = {year} and month={month} and day={day} and hour={hour}")
        myresult = mycursor.fetchall()
        return myresult[0][0]

In [ ]:
hour = start_course.hour
day  = start_course.day
month= start_course.month
year = start_course.year
date = start_course.strftime("%Y-%m-%d %H:%M:%S")
id_time=add_time(hour,day,month,year,date)
id_time

In [ ]:
#ADD PROFESSOR FROM LOG TO TABLE
for index, row in df_log.iterrows():
    if row["Rôle"]=="Organisateur":
        name,email = row["Nom complet"],row["E-mail"]
        id_prof = add_professor(name,email)
        print("professor:",id_prof)

In [ ]:
groups = df_log.groupby("E-mail")

In [ ]:
mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
mycursor = mydb.cursor()
mycursor.execute(f"SELECT * FROM students")
myresult = mycursor.fetchall()
emails=[]
for result in myresult:
    emails.append(result[3])

In [ ]:
print((end_course-start_course).total_seconds() / 60.0)

In [ ]:

for email in emails:
    name,gender,email,course,major = None,None,email,course,None
    id_student=add_student(name,gender,email,course,major)
    try:
        data = groups.get_group(email).reset_index()
        nb_abs = 0
        #DURATION
        duration=0
        got_first_start=False
        for index, row in data.iterrows():
            start_str = row["Heure d'arrivée"].replace(" à","")
            end_str   = row["Heure de départ"].replace(" à","")
            start = datetime.datetime.strptime(start_str, '%d/%m/%Y %H:%M:%S')
            end = datetime.datetime.strptime(end_str, '%d/%m/%Y %H:%M:%S')
            duration = duration + ((end-start).total_seconds() / 60.0)
            if not got_first_start:
                got_first_start = True
                first_start = start
        if duration > 120:
            duration=120
        #NB LATES  
        if ((first_start-start_course).total_seconds() / 60.0) > 15:
            nb_lates = 1
        else:
            nb_lates = 0
        
        nb_pres = 1
        
        #ABS DURATION
        abs_duration = 120-duration
        
        #DURATION IN INTEGER IN DB!
        duration = int(duration)
        
        #is student
        print("student:",id_student)
        if duration <= 15:
            nb_abs = 1
            nb_lates=0
            nb_pres=0
    
    except KeyError as e:
        print("not found",e)
        nb_abs = 1
        abs_duration = 120
        nb_lates=0
        nb_pres=0
        duration=0
    #insert to data mart now
    try:
        mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
        mycursor = mydb.cursor()
        sql = "INSERT INTO fact_table (id_time,id_prof,id_student,nb_abs,abs_duration,nb_lates,nb_pres,duration) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
        val = (id_time,id_prof,id_student,nb_abs,abs_duration,nb_lates,nb_pres,duration)
        mycursor.execute(sql, val)
        mydb.commit()
    except:
        continue

In [ ]:
os.system(f'mv "{input_file}" "/home/anas/notebook/UseCase/processed/"')